This example explains how to build a custom noise model for a circuit.

In [15]:
from matchgates.generators import XY_circuit
from matchgates import SingleGateNoise, MatchGate, NoisyAppliedMatchGate, NoisyCircuit
import numpy as np

The first step is to generate a SingleGateNoise object, which is a noise channel whose kraus maps are scaled matchgates. These are specified by a list of pairs of a MatchGate and a number p (the probability of that matchgate noise being applied). In addition to this an additional number must also be given which is the probability of the identity occuring.
Here is an example:

In [16]:
noise1_params = MatchGate.empty_param_dict()
noise1_params["xx"] = -0.5
noise1_params["z1"] = -0.2
noise2_params = MatchGate.empty_param_dict()
noise2_params["xy"] = -0.1

noise1 = MatchGate(noise1_params)
noise2 = MatchGate(noise2_params)

id_prob = 0.4  # Probability of the identity occuring (i.e. no noise!)
noise = SingleGateNoise([(noise1, 0.3), (noise2, 0.3)], id_prob)

To apply this model to a circuit, there are two options:

1. If all two qubit gates in the circuit have the same noise model then it is possible to add this noise model to a circuit directly using the add_two_qubit_uniform_noise method.
2. Otherwise, specify the noise model of every gate in the circuit and build a NoisyCircuit object from this.

We illustrate case 1. first:

In [17]:
n_qubits = 10  # number of qubits
dt = 0.1  # trotter step time interval
J = 0.5  # coupling strength
h = 0.23  # tranverse field
trotter_steps = 5  # number of trotter steps

# generate the XY and Ising circuits.
xy = XY_circuit(n_qubits=n_qubits, dt=dt, J=J, h=h, trotter_steps=trotter_steps)

noisy_xy = xy.add_two_qubit_uniform_noise(noise)  # returns a NoisyCircuit object!

If instead, one wants to simulate a matchgate circuit with non-uniform noise model, it is necessary to insert add the desired noise model to every individual gate, as follows:

In [18]:
depol_noise = SingleGateNoise.matchgate_depolarizing(
    0.1
)  # a different single qubit noise model.

noisy_gates = []
for gate in xy.gate_list:
    if np.isclose(gate.mg.param_dict["xx"], 0, atol=1e-8):
        noisy_gate = NoisyAppliedMatchGate(
            gate, noise
        )  # We apply the custom built noise model to all the transverse fields.
    else:
        noisy_gate = NoisyAppliedMatchGate(
            gate, depol_noise
        )  # And we apply the matchgate-depolarizing noise to the 2-qubit gates.
    noisy_gates.append(noisy_gate)

noisy_circuit = NoisyCircuit(
    xy.n_qubits, noisy_gates
)  # Construct a NoisyCircuit object from the noisy gates.

Now that we have constructed a NoisyCircuit object using a custom noise model, it is possible to simulate this by following the example in `examples/noisy_circuits.ipynb`